```{contents}
```
## **Function Node in LangGraph**

A **Function Node** in LangGraph is the fundamental **deterministic computation unit** of a graph.
It executes **pure Python logic** over the shared state and returns a **partial state update** that drives subsequent execution.

Function nodes are used for:

* control logic
* validation
* transformation
* routing
* tool orchestration
* safety checks
* business rules

They form the **algorithmic backbone** of any production LangGraph system.

---

### **1. Core Intuition**

In LangGraph:

> **State flows through nodes.
> Each node reads state, computes something, and returns updates.**

A Function Node is simply:

```python
def node_fn(state) -> dict:
    return {...}   # state update
```

It is **stateless itself** and derives all behavior from the input state.

---

### **2. Formal Definition**

| Property     | Description                           |
| ------------ | ------------------------------------- |
| Input        | Entire current state                  |
| Output       | Partial state update (dict)           |
| Side Effects | Optional (API calls, DB writes, etc.) |
| Determinism  | Yes (unless external side effects)    |
| Execution    | Synchronous by default                |
| Type         | Pure Python function                  |

---

### **3. Basic Example**

```python
def normalize_text(state):
    text = state["input"].lower().strip()
    return {"normalized": text}
```

```python
builder.add_node("normalize", normalize_text)
```

---

### **4. How Function Nodes Modify State**

LangGraph merges returned values into the global state:

```python
state = {"input": "  Hello "}
update = normalize_text(state)

# New state:
# {"input": "  Hello ", "normalized": "hello"}
```

This makes function nodes **safe, composable, and testable**.

---

### **5. Function Nodes in Control Flow**

Function nodes often act as **routers**.

```python
def route(state):
    if state["confidence"] > 0.8:
        return "accept"
    return "revise"
```

```python
builder.add_conditional_edges("route", route, {
    "accept": "final",
    "revise": "review"
})
```

---

### **6. Using Function Nodes for Validation**

```python
def validate(state):
    errors = []
    if len(state["query"]) < 5:
        errors.append("Query too short")
    return {"errors": errors}
```

---

### **7. Function Node with External Systems**

```python
def save_to_db(state):
    db.insert(state["result"])
    return {"saved": True}
```

---

### **8. Async Function Nodes**

```python
async def fetch_data(state):
    data = await api_call(state["url"])
    return {"data": data}
```

```python
builder.add_node("fetch", fetch_data)
```

---

### **9. Production Use Cases**

| Use Case            | Purpose                     |
| ------------------- | --------------------------- |
| Input preprocessing | Clean and structure inputs  |
| Business logic      | Apply rules and policies    |
| Routing             | Control graph flow          |
| Validation          | Safety and quality checks   |
| Tool orchestration  | Decide which tool to call   |
| Error handling      | Detect and recover failures |
| Metrics logging     | Capture performance data    |

---

### **10. Function Node vs LLM Node**

| Feature     | Function Node | LLM Node  |
| ----------- | ------------- | --------- |
| Determinism | Yes           | No        |
| Cost        | Free          | Expensive |
| Latency     | Low           | High      |
| Reliability | High          | Variable  |
| Testing     | Easy          | Harder    |

**Production systems maximize function nodes and minimize LLM calls.**

---

### **11. Design Best Practices**

* Keep function nodes **small and single-purpose**
* Make nodes **idempotent**
* Avoid long blocking operations
* Log inputs and outputs
* Handle failures explicitly
* Prefer function nodes for control and validation

---

### **12. Minimal Complete Graph Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    input: str
    normalized: str
    result: str

def normalize(state):
    return {"normalized": state["input"].lower()}

def process(state):
    return {"result": state["normalized"][::-1]}

builder = StateGraph(State)
builder.add_node("normalize", normalize)
builder.add_node("process", process)

builder.set_entry_point("normalize")
builder.add_edge("normalize", "process")
builder.add_edge("process", END)

graph = builder.compile()
print(graph.invoke({"input": "HELLO"}))
```

---

### **Conceptual Summary**

A Function Node is the **deterministic brain** of a LangGraph workflow:
it enforces logic, structure, safety, and performance while LLM nodes provide intelligence.


### Demonstration

In [1]:
from typing import TypedDict

class State(TypedDict):
    raw: str
    normalized: str
    valid: bool
    result: str


def normalize(state: State):
    return {"normalized": state["raw"].strip().lower()}

def validate(state: State):
    ok = len(state["normalized"]) >= 5
    return {"valid": ok}

def router(state: State):
    if state["valid"]:
        return "process"
    return "reject"

def process(state: State):
    return {"result": f"Processed: {state['normalized']}"}

def reject(state: State):
    return {"result": "Input too short"}

In [2]:
from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("normalize", normalize)
builder.add_node("validate", validate)
builder.add_node("process", process)
builder.add_node("reject", reject)

builder.set_entry_point("normalize")
builder.add_edge("normalize", "validate")

builder.add_conditional_edges("validate", router, {
    "process": "process",
    "reject": "reject"
})

builder.add_edge("process", END)
builder.add_edge("reject", END)

graph = builder.compile()


In [3]:
result = graph.invoke({"raw": "   Hello World   "})
print(result)


{'raw': '   Hello World   ', 'normalized': 'hello world', 'valid': True, 'result': 'Processed: hello world'}
